In [4]:
import pandas as pd
import os 
import pathlib
import numpy as np 
import matplotlib.pyplot as plt

In [7]:
def openPsychoPyExperimentMetadata(path):
    df = pd.read_csv(path)
    df = df[~df['videos'].isna()]
    df = df[['videos','participant','sessions.thisN','trial.started','trial.stopped']]
    df.columns.values[2] = 'session'
    df['participant'] = df['participant'].astype(int)
    df['session'] = df['session'].astype(int)
    df['videos'] = '../' + df['videos']
    return df

def buildOneHotEncoding(df):
    filepath = df['videos']
    filepath = filepath.split('/')[2:-1]
    filepath[0] = filepath[0].split('_')[1:]
    df['init'] = int(filepath[0][0][-1])
    df['perturbation'] = 1 if filepath[0][-1] == "perturbation" else 0
    df["delta"] = 0 if filepath[-1] == "original" else int(filepath[-1][-1])
    
    return df

In [8]:
df = openPsychoPyExperimentMetadata("../data/188563_PFE_2024-01-29_18h07.46.466.csv")
df = df.apply(lambda x : buildOneHotEncoding(x), axis = 1)
t0 = df['trial.started'][0]
df['trial.started'] = df.apply(lambda x : x['trial.started'] - t0 , axis  = 1 )
df['trial.stopped'] = df.apply(lambda x : x['trial.stopped'] - t0 , axis  = 1 )
df.head(-1)

,videos,participant,session,trial.started,trial.stopped,init,perturbation,delta
0,../video/Sphere_init1_speed1/delta1/0001-0150.avi,188563,0,0.000000,6.333386,1,0,1
1,../video/Sphere_init1_speed1/delta2/0001-0150.avi,188563,0,7.450369,13.950540,1,0,2
2,../video/Sphere_init2_speed1_perturbation/delt...,188563,0,15.066999,21.550252,2,1,2
3,../video/Sphere_init3_speed1/delta2/0001-0150.avi,188563,0,22.683611,29.166660,3,0,2
4,../video/Sphere_init2_speed1_perturbation/orig...,188563,0,30.300306,36.783160,2,1,0
...,...,...,...,...,...,...,...,...
68,../video/Sphere_init1_speed1_perturbation/delt...,188563,2,510.916258,517.399522,1,1,2
69,../video/Sphere_init2_speed1_perturbation/delt...,188563,2,518.532922,525.016250,2,1,2
70,../video/Sphere_init4_speed1/original/0001-015...,188563,2,526.149092,532.632803,4,0,0
71,../video/Sphere_init3_speed1_perturbation/delt...,188563,2,533.749647,540.249436,3,1,2


In [9]:
gaze_positions = pd.read_csv('../pupil_data/gaze_positions.csv')
t0 = gaze_positions.gaze_timestamp[0]
gaze_positions.gaze_timestamp = gaze_positions.apply(lambda x : x.gaze_timestamp - t0, axis = 1)
gaze_positions.head()

,gaze_timestamp,world_index,confidence,norm_pos_x,norm_pos_y,base_data,gaze_point_3d_x,gaze_point_3d_y,gaze_point_3d_z,eye_center0_3d_x,...,eye_center0_3d_z,gaze_normal0_x,gaze_normal0_y,gaze_normal0_z,eye_center1_3d_x,eye_center1_3d_y,eye_center1_3d_z,gaze_normal1_x,gaze_normal1_y,gaze_normal1_z
0,0.000000,0,0.983184,0.292788,0.327241,433986.852817-0 433986.853037-1,-215.248987,72.589447,624.976080,19.980748,...,-20.161165,-0.341332,0.091532,0.935476,-40.038998,15.031599,-19.912736,-0.261225,0.077989,0.962122
1,0.004015,0,0.973617,0.292013,0.328058,433986.860848-0 433986.853037-1,-211.859581,70.691415,612.570811,19.980748,...,-20.161165,-0.342851,0.089134,0.935151,-40.038998,15.031599,-19.912736,-0.261225,0.077989,0.962122
2,0.008033,0,0.990433,0.291990,0.329240,433986.860848-0 433986.861072-1,-212.198999,70.099421,613.547185,19.980748,...,-20.161165,-0.342851,0.089134,0.935151,-40.038998,15.031599,-19.912736,-0.261352,0.075969,0.962249
3,0.012034,0,1.000000,0.292834,0.330886,433986.868851-0 433986.861072-1,-215.945740,70.643342,627.390119,19.980748,...,-20.161165,-0.341190,0.087265,0.935935,-40.038998,15.031599,-19.912736,-0.261352,0.075969,0.962249
4,0.016026,0,1.000000,0.292513,0.330621,433986.868851-0 433986.86905599997-1,-222.847148,72.949834,646.282342,19.980748,...,-20.161165,-0.341190,0.087265,0.935935,-40.038998,15.031599,-19.912736,-0.263705,0.077843,0.961457


In [170]:
def retrieve_gaze_positions(df):
    gaze = gaze_positions[(gaze_positions['gaze_timestamp']>=df['trial.started']) & (gaze_positions['gaze_timestamp']<df['trial.stopped'])]
    df['pos_x'] = gaze['norm_pos_x'].tolist()
    df['pos_y'] =  gaze['norm_pos_y'].tolist()
    
    return df
    

df = df.apply(lambda x : retrieve_gaze_positions(x) , axis = 1 )
df.head()

,videos,participant,session,trial.started,trial.stopped,init,perturbation,delta,pos_x,pos_y
0,../video/Sphere_init1_speed1/delta1/0001-0150.avi,188563,0,0.000000,6.333386,1,0,1,"[0.2927884583813753, 0.2920131246781436, 0.291...","[0.3272405972755005, 0.3280579570834274, 0.329..."
1,../video/Sphere_init1_speed1/delta2/0001-0150.avi,188563,0,7.450369,13.950540,1,0,2,"[0.3221923869596849, 0.4911111843027094, 0.297...","[0.4269767361632504, 0.2522769067476786, 0.335..."
2,../video/Sphere_init2_speed1_perturbation/delt...,188563,0,15.066999,21.550252,2,1,2,"[0.3303922956368795, 0.3307511300507559, 0.331...","[0.1916913822992578, 0.1906563279423665, 0.191..."
3,../video/Sphere_init3_speed1/delta2/0001-0150.avi,188563,0,22.683611,29.166660,3,0,2,"[0.5856729326233265, 0.5865108178014103, 0.586...","[0.3165234793660445, 0.3187832786146326, 0.317..."
4,../video/Sphere_init2_speed1_perturbation/orig...,188563,0,30.300306,36.783160,2,1,0,"[0.2663852177642082, 0.1802998259758378, 0.299...","[0.067885146185584, -0.0466315406006683, 0.047..."
